# Building a Data Dashboard with Deno and Jupyter Notebooks
To help you understand how the new Deno kernel works, it's better if we do a basic data visualization exercise using Jupyter Notebooks and Deno. 
You will build a financial data dashboard that analyzes your income and expenses. This dashboard will provide visualizations and insights on your financial data, helping you understand your spending habits overtime and make informed financial decisions. It will also allow you to track your net income, and identify trends.
Here is a sample dataset of what the data would look like:

```
Date,Income,Expenses,NetIncome,BudgetIncome,ActualIncome,BudgetExpenses,ActualExpenses,Salaries,R&D,Marketing,Utilities,Rent,Equipment,Software,Hardware,Consulting,Office Supplies,DiffIncome,DiffExpenses
2022-01-01,281,218,63,284,281,221,218,41,24,45,43,22,35,2,2,2,2,3,3
2022-01-02,328,244,84,323,328,240,244,46,45,34,35,31,37,1,4,8,3,-5,-4
2022-01-03,266,223,43,269,266,222,223,31,49,38,30,22,40,2,6,1,4,3,-1
2022-01-04,287,226,61,279,287,229,226,43,47,31,48,21,26,5,1,3,1,-8,3
2022-01-05,307,214,93,309,307,217,214,48,37,40,23,34,20,1,3,4,4,2,3
```
The dataset for the dashboard is publicly available here.

## Choosing the Right Tools for Data Visualization
Choose the tools you're going to use the following packages:
- skia_canvas
- display
- d3.js
- @observablehq/plot
`skia_canvas` will be used to create a canvas for rendering the data visualizations, `display` will be used to render the plot on the canvas, `d3.js` will be used for data visualization, and `@observablehq/plot` is a JavaScript library for creating interactive plots.

## Start Building the Data Visualization Dashboard
To build the financial data dashboard, you can start by importing the necessary libraries and modules in your Jupyter Notebook. Begin by importing the `nodejs-polars` library, as well as the Display module. These modules and libraries will be used for processing the data for your dashboard.

In [3]:
import pl from "npm:nodejs-polars";
import { display } from "https://deno.land/x/display@v0.1.1/mod.ts";

Once you have the required libraries imported, you can start fetching the sample dataset before the data visualization process. 

In [ ]:
let response = await fetch(
"https://gist.githubusercontent.com/agustinustheo/195f32a4a6c68c493056c883d959ca35/raw/c7363d8b916ab00a2d1747adb89fca120da29f42/mock_financial_data.csv",
);

let data = await response.text();

let df = pl.readCSV(data, { sep: "," });

await display(df.sample(10));

In [4]:
import * as d3 from "npm:d3";
import { createCanvas } from "https://deno.land/x/skia_canvas/mod.ts";

In [ ]:
let categories = ['Salaries', 'R&D', 'Marketing', 'Utilities', 'Rent', 'Equipment', 'Software', 'Hardware', 'Consulting', 'Office Supplies'];

const lastDataPoint = df.tail(1).toRecords()[0];
console.log(lastDataPoint);

// Sample data
const sampleData1 = [];
for(let i = 0; i < categories.length; i++) {
    const category = categories[i];
    sampleData1.push({
        category,
        amount: lastDataPoint[category],
    });
}

In [ ]:
const width = 500;
const height = 500;
const radius = Math.min(width, height) / 2;

// Create a pie function
const pie = d3.pie().value(d => d.amount);

// Create an arc generator for the slices
const arc = d3.arc()
    .innerRadius(0)
    .outerRadius(radius);

// Create an arc generator for the labels
const labelArc = d3.arc()
    .innerRadius(radius - 40) // Adjust to position the labels
    .outerRadius(radius - 40);

// Create the canvas
const canvas = createCanvas(width, height);
const ctx = canvas.getContext("2d");

// Translate to center the pie chart
ctx.translate(width / 2, height / 2);

// Draw the pie chart
pie(sampleData1).forEach((d, i) => {
    // Draw the slice
    ctx.beginPath();
    arc.context(ctx)(d);
    ctx.fillStyle = d3.schemeCategory10[i % 10];
    ctx.fill();

    // Draw the label
    ctx.fillStyle = "#000"; // Label color
    ctx.textAlign = "center";
    ctx.textBaseline = "middle";
    
    const centroid = labelArc.centroid(d);
    ctx.fillText(d.data.category, centroid[0], centroid[1]);
});

// Display the canvas
await display(canvas);

In [ ]:
import * as Plot from "npm:@observablehq/plot";
import { DOMParser, SVGElement } from "npm:linkedom";
const document = new DOMParser().parseFromString(
  `<!DOCTYPE html><html lang="en"></html>`,
  "text/html",
);

let records = df.tail(40).toRecords();

In [ ]:
let convertedArray = records.map(item => {
    return {
        NetIncome: item.NetIncome,
        Date: new Date(item.Date)
    };
});

await display(
  Plot.plot({
    x: { type: "band" },
    y: { grid: true },
    marks: [
      Plot.line(convertedArray, {x: "Date", y: "NetIncome"}),
    ],
    document,
  }),
);

In [ ]:


let sampleData2 = [];
for (let i = 0; i < records.length; i++) {
  const currentRecord = records[i];
  for (let x = 0; x < categories.length; x++) {
    const currentCategory = categories[x];
    sampleData2.push({
      date: new Date(currentRecord["Date"]),
      category: currentCategory,
      count: currentRecord[currentCategory],
    });
  }
}

await display(
  Plot.plot({
    x: { type: "band" },
    y: { grid: true },
    marks: [
      Plot.barY(sampleData2, { x: "date", y: "count", fill: "category" }),
      Plot.ruleY([0]),
    ],
    color: { legend: true },
    document,
  }),
);